# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 2)

## Introduction: 
Coffee has evolved from a luxury in the past to a commmondity - to some a need in the world today

Quoting it has been that over 2.25 billion cups of coffee are consumed in the world every day worldwide. Coffee has been a important part of human live's, be it for its caffine, aroma, or other functions. Nonetheless, it might have also become and international too for conneting people

## Problem:
Since the early 2010s, cafe busines has been a strong interest till date. As such, this project will try recommend new cafe ventureses to select ideal locations to set up cafes by
- Location population 
- Surrounding businesses concerns
around region of Toronto, Canada 

## Location
Toronto, Canada

## Data 
The following datas will be acquired
- Postal codes of disctricts/regions in Toronto
- Point of Interest (POI)s or establishments in the regions using clustering

The regions/disctrict names wil be atttined by using wikipedia , while the clustering and mapping be done using foursquare API.

Approach: The datas will allow us to obtain clusters, and from the clusters to decide if the locations are likely idea for a cafe business to strive there. Analysis will be more geographical apporach than evalutation of the k-clusters; as such only observations can be provided, not full recommendations

## Methodology 

###  - Obtaining dataset

In [1]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!pip install geopy
from geopy.geocoders import Nominatim

import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!pip install folium # uncomment this line if you haven't completed the Foursquare API lab
import folium 

In [5]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
dat = table[0]
dat2= dat.drop(dat[dat['Borough'] == 'Not assigned'].index)

dat2.Neighborhood = dat2.Borough.where(dat2.Neighborhood == 'Not assigned', dat2.Neighborhood)
dat3 = dat2.groupby(['Postal Code','Borough']).agg(lambda col: ','.join(col)).reset_index()

In [6]:
geoid = pd.read_csv("http://cocl.us/Geospatial_data")
geoid2 = geoid.sort_values(by=['Postal Code'])
geodata = pd.merge(dat3,geoid2,on  = 'Postal Code', how = 'right')
toronto_data = geodata[geodata['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


### - Obtain cooridnates of Toronto, CA for mapping focus

In [7]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto,CA are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto,OH are 43.6534817, -79.3839347.


In [10]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            'XXIUY2LRH23D5M4UCRA0G4IBSATEC3B3CA54ZTM5BX3RHGST', 
            'RYVSQEKBBZJHUO5KGFEGKSR1F1FPKDJQ40XS0MFAATLQUF1Y', 
            '20180605', 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [22]:
# one hot encoding
toronto_hot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_hot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = toronto_hot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Neighborhood')))

#then use .reindex() function to reorder
toronto_hot = toronto_hot.reindex(columns= cols)
toronto_grouped = toronto_hot.groupby('Neighborhood').mean().reset_index()

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:

# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 8, 7, 1, 9, 1, 1, 8, 8], dtype=int32)

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,5,Trail,Asian Restaurant,Health Food Store,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place,Lounge
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,8,Sandwich Place,Park,Pizza Place,Steakhouse,Ice Cream Shop,Fish & Chips Shop,Sushi Restaurant,Light Rail Station,Pub,Italian Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Gastropub,Brewery,American Restaurant,Fish Market,Clothing Store,Stationery Store,Food
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Photography Studio,Bus Line,Swim School,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,8,Park,Pizza Place,Breakfast Spot,Hotel,Sandwich Place,Food & Drink Shop,Department Store,Gym / Fitness Center,Gym,Dumpling Restaurant
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,1,Clothing Store,Coffee Shop,Yoga Studio,Chinese Restaurant,Spa,Diner,Sporting Goods Shop,Mexican Restaurant,Fast Food Restaurant,Café
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,8,Dessert Shop,Café,Sandwich Place,Pizza Place,Sushi Restaurant,Italian Restaurant,Gym,Coffee Shop,Tennis Court,Farmers Market
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,6,Trail,Park,Gym,Tennis Court,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,1,Coffee Shop,Pub,Pizza Place,Sushi Restaurant,Bank,Restaurant,Bagel Shop,Fried Chicken Joint,Sports Bar,Supermarket


### - Data above revelas that the top 10 POIs for each neighborhood

In [41]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='rainbow[cluster-1]',
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Legends -

Cluster 1 = Purple

Cluster 2 - Dark 

Cluster 3 = Blue

Cluster 4 = Light Blue

Cluster 5 = Green

Cluster 6 = Cyan


Cluster 8 = Beige

Cluster 9 = Blood orange

Cluster 0 = Red


### Results

- Café and Coffee shops has one of the highest frequency appearing in the table of top 10, which implies that coffee culture in Toronto is strong and ideal for venturing into the coffee business
- Based on the clustering outcome, setting up a business in District 1 is ideal, especially near the Toronto downtown  
- However, reantals around downtown is expected to be high so low risk ventureres can try the region bettween cluster 2, 4 ,8 centered at Elington

### Discussion

- The mapping and clustering has it limitations as there are other factors to be considerd, such as business legistratives and standard of living
- Business trends are always changing, so such a one-time off analysis is not sufficeint enought to band it in a good confiedence interval of expecation

### Conclusion 

Toronto has been shown to be an ideal location for cafe business based on the data's revalation of the populairty of cafes and coffee shops, however there are more indept business constrains and concerned to be considred.
A more indept regression study will be needed to have a better prediction and a dispaly of CBA (Cost Benefit Analsysis